In [1]:
# RUN JUST ONCE TO DOWNLOAD THE DRIVER
from webdriver_manager.chrome import ChromeDriverManager

# Chrome driver installation
CHROMEDRIVER_PATH = ChromeDriverManager().install()

print(CHROMEDRIVER_PATH) # Chrome driver path

In [235]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def start_driver():
    """ Initialize Chrome driver and open main page """
    
    # Threads URL
    URL = "https://www.threads.net/?hl=en"
    # Chrome driver PATH
    CHROMEDRIVER_PATH = r"C:\Users\lucal\.wdm\drivers\chromedriver\win64\131.0.6778.85\chromedriver-win32/chromedriver.exe"
    
    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH))
    
    # # Navigate to the URL + explicit wait
    driver.get(URL)
    time.sleep(2)
    
    return driver

In [236]:
# Import the By class for locating elements
from selenium.webdriver.common.by import By
# Import wait and EC to form an implicit wait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# timeout exception
from selenium.common.exceptions import TimeoutException

def accept_cookies(driver):
    """ accept coookies """

    # CSS SELECTOR
    ACCEPT_COOKIE_SELECTOR = 'body > div.__fb-dark-mode.x1n2onr6.xzkaem6 > div:nth-child(2) > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div.x1uvtmcs.x4k7w5x.x1h91t0o.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1n2onr6.x1qrby5j.x1jfb8zj > div > div > div > div.x6s0dn4.x78zum5.xl56j7k.x1n2onr6.x1vjfegm > div > div > div > div.x6bh95i.x13fuv20.x178xt8z.x1p5oq8j.xxbr6pl.xwxc41k.xbbxn1n > div > div.x1i10hfl.x1ypdohk.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xexx8yu.x18d9i69.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1q0g3np.x1lku1pv.x1a2a7pz.x6s0dn4.x1a2cdl4.xnhgr82.x1qt0ttw.xgk8upj.x9f619.x3nfvp2.x1s688f.x90ne7k.xl56j7k.x193iq5w.x1swvt13.x1pi30zi.x12w9bfk.x1g2r6go.x11xpdln.xz4gly6.x87ps6o.xuxw1ft.x19kf12q.x111bo7f.x1vmvj1k.x45e8q.x3tjt7u.x35z2i1.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x178xt8z.xm81vs4.xso031l.xy80clv.xteu7em.x11r8ahe.x1iyjqo2.x15x72sd > div'
    # XPATH
    ACCEPT_COOKIE_XPATH = "//div[contains(text(), 'Allow all cookies')]"

    # TIMEOUT (15 seconds)
    TIMEOUT = 15
    
    try:
        # implicit wait
        button = WebDriverWait(driver, TIMEOUT).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ACCEPT_COOKIE_SELECTOR)) # with CSS_SELCTOR
        )
        # click to accept cookies
        button.click()
        print("CSS cookies button click\n")
    
    except TimeoutException:
        try: 
            button = WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.XPATH, ACCEPT_COOKIE_XPATH)) # with XPATH
            )
            button.click()
            print("XPTAH cookies button click\n")

        except TimeoutException:    
            print("ERROR: Cookies button missing")
            driver.quit() # cleaner termination ensuring a clean state for future executions

    

In [237]:
from selenium.common.exceptions import NoSuchElementException

def grab_posts(driver):
    """ Grab posts and return a list of dict """

    # class to handle each post
    class Post:
        def __init__(self, imgProfile, username, username_link, post_link):
            self._img_profile = img_profile
            self._username = username
            self._username_link = username_link
            self._post_link = post_link
        
        #getter
        @property
        def single_post(self):
            return {
                "img_profile": self._img_profile, 
                "username": self._username,
                "username_link": self._username_link,
                "post_link": self._post_link
                }
    
    list_of_posts = []

    '''
    POST CONTAINER RAW XPATH:
    POST_XPATH 1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div
    POST_XPATH 2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div
    POST_XPATH 3RD POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[3]/div/div/div/div
    
    IMAGE RAW XPATH:
    1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[1]/div/a/div/img
    2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[1]/div/a/div/img
    
    USERNAME RAW XPATH:
    1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a/span/
    2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a/span/

    USERNAME LINK RAW XPATH:
    1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a
    2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a

    POST LINK RAW XPATH:
    1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[2]/div/div[1]/div/span[2]/span/a
    2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div[1]/div/span[2]/span/a
    '''
 
    POST_XPATH = '//*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[*]/div/div/div/div'
    
    IMAGE_PROFILE_REL_XPATH = './/div[1]/div/a/div/img'
    USERNAME_REL_XPATH      = './/div[2]/div/div[1]/div/span[1]/div/div/a/span'
    USERNAME_LINK_REL_XPATH = './/div[2]/div/div[1]/div/span[1]/div/div/a'
    POST_LINK_REL_XPATH     = './/div[2]/div/div[1]/div/span[2]/span/a'
    
    # retrieve all posts in a page
    posts = driver.find_elements(By.XPATH, POST_XPATH)
    print(f"Found {len(posts)} posts.")
    
    # for each post build the class and add element to list
    for post in posts:
        try:
            # image link
            img_profile = post.find_element(By.XPATH, IMAGE_PROFILE_REL_XPATH).get_attribute('src') 
            # username visible text
            username = post.find_element(By.XPATH, USERNAME_REL_XPATH).text
            # username link profile
            username_link = post.find_element(By.XPATH, USERNAME_LINK_REL_XPATH).get_attribute('href')
            # post content
            post_link = post.find_element(By.XPATH, POST_LINK_REL_XPATH).get_attribute('href')
            
            # new instance of the class Post
            new_post = Post(img_profile, username, username_link, post_link)
            
            # add new post to list
            list_of_posts.append(new_post.single_post)
        
        # incomplete post
        except NoSuchElementException:
            print("NoSuchElementException")
            continue  

        # other exception
        except Exception as e:
            print("Exception in grab_posts function:", e)
            continue

    # show username for each post collected
    for i, post in enumerate(list_of_posts, start=1):
        print(f"post #{i}: {post}")

In [238]:
def scroll(driver, px):
    """ scroll page """
    
    js = f"window.scrollBy(0, {px});"
    driver.execute_script(js)

In [239]:
from selenium.common.exceptions import InvalidSessionIdException

def close_session(driver):
    """ close active browser tab """
    
    try:
        driver.close() # exit driver
    except InvalidSessionIdException:
        print("Browser tab already closed!")

In [240]:
def main():
    """ scraper main """
            
    # start web driver
    driver = start_driver() 
    # accept initial cookies
    accept_cookies(driver)
    
    print("Website:", driver.title) # fetches the title of the current webpage
    print("Current URL:", driver.current_url) # capture the final URL after all redirects have been resolved

    # wait for the page to load (15 posts)
    time.sleep(15)
    # grab posts 1
    grab_posts(driver)

    # scroll page
    scroll(driver, 4000)
    # wait for the page to load (15 posts)
    time.sleep(15)
    # grab posts 2
    grab_posts(driver)

    time.sleep(5)
    close_session(driver)

In [241]:
##############################################
############ RUN THE SCRAPER #################
##############################################

main()

CSS cookies button click

Website: Threads
Current URL: https://www.threads.net/?hl=en
Found 15 posts.
post #1: {'img_profile': 'https://scontent.cdninstagram.com/v/t51.2885-19/358032426_1388919248338494_7799117333404429295_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent.cdninstagram.com&_nc_cat=106&_nc_ohc=zi3Ks7G-xhIQ7kNvgFi0ib2&_nc_gid=a9bb0a57185f4c628c418fd76453514b&edm=APs17CUBAAAA&ccb=7-5&oh=00_AYBxjQrlK8mqpxDofhOdXHyBMcfWLRcYt-S-cBJe-3JsPQ&oe=6753C3D1&_nc_sid=10d13b', 'username': 'chi_urbanist', 'username_link': 'https://www.threads.net/@chi_urbanist', 'post_link': 'https://www.threads.net/@chi_urbanist/post/DDC4ZqsPd46'}
post #2: {'img_profile': 'https://scontent.cdninstagram.com/v/t51.2885-19/275003278_1155948288485591_8352220700957319862_n.jpg?stp=dst-jpg_s150x150_tt6&_nc_ht=scontent.cdninstagram.com&_nc_cat=111&_nc_ohc=fdW0Hy-KbLcQ7kNvgFwoY_E&_nc_gid=a9bb0a57185f4c628c418fd76453514b&edm=APs17CUBAAAA&ccb=7-5&oh=00_AYAJBZP_loH2gkKTZ0yp6A5IXOaa7pLjlWVqDcWHdqkGiA&oe=6753C629&_nc_sid

In [18]:
# ENV VARIABLES
# https://saturncloud.io/blog/how-to-set-environment-variables-in-jupyter-notebooks-a-guide-for-data-scientists/

# add driver settings
# cookise
# headers
# proxy or VPN
# beware honeypot